# Symmetric Funnel Decomposition Analysis - Weekly Data Demo

This notebook demonstrates the **symmetric (order-independent)** funnel decomposition analysis using **weekly data**.

## Key Features

- **Weekly granularity**: 104 weeks of data (2 years)
- **Symmetric methodology**: Order-independent midpoint approach
- **Same structure**: All dimensions and metrics as monthly data
- **NEW: date_column parameter**: No need to rename date columns!

## Workflow
1. Load weekly booking funnel data
2. Calculate symmetric decomposition effects using `date_column` parameter
3. Visualize results with waterfall charts
4. View detailed breakdowns by dimension

In [ ]:
import sys
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt

# Add src to path
sys.path.insert(0, str(Path.cwd().parent / 'src'))

import symmetric_decomposition_calculator
import visualization_engine

# Configure matplotlib for inline display
%matplotlib inline
plt.rcParams['figure.figsize'] = (16, 12)
plt.rcParams['figure.dpi'] = 100

## 1. Load Weekly Mock Data

The dataset contains 104 weeks (2 years) of booking funnel data with:
- **Dimensions**: FICO bands (High/Med/Low/Null), offer competition tier, product line
- **Funnel metrics**: Applications, approval rates, booking rates
- **Segments**: 24 segments per week (4 FICO bands × 3 offer tiers × 2 product lines)
- **Frequency**: Weekly (every Monday)

In [ ]:
# Load weekly data
data_path = Path.cwd().parent / 'data' / 'funnel_data_mock_weekly.csv'
df = pd.read_csv(data_path)
df['week_begin_date'] = pd.to_datetime(df['week_begin_date'])

print(f"Loaded {len(df)} rows of data")
print(f"\nDate range: {df['week_begin_date'].min().date()} to {df['week_begin_date'].max().date()}")
print(f"Unique weeks: {df['week_begin_date'].nunique()}")
print(f"Segments per week: {len(df) // df['week_begin_date'].nunique()}")

# Show sample data
print("\nSample data (first week):")
df[df['week_begin_date'] == df['week_begin_date'].min()].head(10)

## 2. Select Weeks for Analysis

**NEW in this version**: We no longer need to rename the date column! 

The calculator now supports a `date_column` parameter that lets you specify which date column to use directly. This keeps your data clean and makes the code more explicit about what you're analyzing.

In [ ]:
# Show available weeks for analysis
unique_weeks = sorted(df['week_begin_date'].unique())
print(f"Total weeks available: {len(unique_weeks)}")
print(f"\nFirst 10 weeks:")
for i, week in enumerate(unique_weeks[:10], 1):
    print(f"  {i}. {week.date()}")

## 3. Calculate Symmetric Decomposition (Week-over-Week)

Let's compare Week 26 (mid-2023) → Week 78 (mid-2024) to see a full year of changes.

### Symmetric Methodology (Order-Independent)

All effects use **average values** from both periods:

1. **Volume Effect**: `ΔA × p_avg × conversion_avg`
2. **Mix Effect**: `A_avg × Δp × conversion_avg`
3. **Straight Approval Effect**: `A_avg × p_avg × Δr_str × b_str_avg`
4. **Conditional Approval Effect**: `A_avg × p_avg × Δr_cond × b_cond_avg`
5. **Straight Booking Effect**: `A_avg × p_avg × r_str_avg × Δb_str`
6. **Conditional Booking Effect**: `A_avg × p_avg × r_cond_avg × Δb_cond`
7. **Interaction Effect**: Residual ensuring perfect reconciliation

### Using the date_column Parameter

Notice below: we specify `date_column='week_begin_date'` to tell the calculator which column contains our dates. No renaming needed!

In [ ]:
# Select weeks for comparison (Week 26 vs Week 78 = ~1 year apart)
week_1 = unique_weeks[25]  # Week 26 (index 25)
week_2 = unique_weeks[77]  # Week 78 (index 77)

print(f"Comparing:")
print(f"  Week 1 (Period 1): {week_1.date()}")
print(f"  Week 2 (Period 2): {week_2.date()}")
print(f"  Time span: {(week_2 - week_1).days} days (~{(week_2 - week_1).days // 7} weeks)")
print()

# Calculate symmetric decomposition with date_column parameter
results = symmetric_decomposition_calculator.calculate_decomposition(
    df=df,
    date_a=week_1,
    date_b=week_2,
    lender='ACA',
    date_column='week_begin_date'  # NEW: Specify the date column directly!
)

print("Symmetric Decomposition Results:")
print(f"  Method: {results.metadata['method']}")
print(f"  Period 1: {results.metadata['date_a']}")
print(f"  Period 2: {results.metadata['date_b']}")
print(f"  Period 1 bookings: {results.metadata['period_1_total_bookings']:,.0f}")
print(f"  Period 2 bookings: {results.metadata['period_2_total_bookings']:,.0f}")
print(f"  Delta bookings: {results.metadata['delta_total_bookings']:+,.0f}")
print(f"\n  Number of segments: {results.metadata['num_segments']}")

## 4. View Summary Table

The summary shows aggregate impact of each effect using the symmetric (order-independent) methodology:

In [ ]:
results.summary

## 5. Create Waterfall Grid

The waterfall grid shows:
- **Top-left**: Overall aggregate waterfall
- **Top-right**: Breakdown by FICO bands (High/Med/Low/Null)
- **Bottom-left**: Breakdown by Offer Comp Tier (solo/multi_best/multi_other)
- **Bottom-right**: Breakdown by Product Line (Used/VMax)

**Color scheme**:
- Green shades = positive contributions
- Red shades = negative contributions
- Gray = Start/End bars

In [ ]:
# Create waterfall grid
fig = visualization_engine.create_waterfall_grid(
    summary=results.summary,
    segment_detail=results.segment_detail,
    lender='ACA'
)

plt.show()

## 6. Detailed Breakdowns by Dimension

For each dimensional waterfall, we can view the exact contribution of each dimension value:

In [ ]:
# Print detailed breakdowns
visualization_engine.print_waterfall_breakdowns(fig)

## 7. Dimension Drilldown Charts

We can create horizontal bar charts showing the impact of each effect broken down by a specific dimension.

### By FICO Band

In [ ]:
# Create FICO band drilldown
fig_fico = visualization_engine.create_dimension_drilldown(
    segment_detail=results.segment_detail,
    dimension='fico_bands',
    lender='ACA'
)

plt.show()

### By Offer Comp Tier

In [ ]:
# Create Offer Comp Tier drilldown
fig_comp = visualization_engine.create_dimension_drilldown(
    segment_detail=results.segment_detail,
    dimension='offer_comp_tier',
    lender='ACA'
)

plt.show()

### By Product Line

In [ ]:
# Create Product Line drilldown
fig_prod = visualization_engine.create_dimension_drilldown(
    segment_detail=results.segment_detail,
    dimension='prod_line',
    lender='ACA'
)

plt.show()

## 8. Segment-Level Detail

For detailed analysis, we can examine the segment-level breakdown:

In [ ]:
# Show segment detail (first 10 segments)
results.segment_detail.head(10)

## 9. Weekly Trend Analysis (Bonus)

With weekly data, we can analyze trends over time:

In [ ]:
# Calculate weekly totals
weekly_totals = df.groupby('week_begin_date').agg({
    'num_tot_apps': 'first',
    'num_tot_bks': 'first'
}).reset_index()

# Create trend plot
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(16, 8))

# Applications trend
ax1.plot(weekly_totals['week_begin_date'], weekly_totals['num_tot_apps'], 
         marker='o', markersize=3, linewidth=1.5, color='#2E86AB')
ax1.set_title('Weekly Applications Trend', fontsize=14, fontweight='bold')
ax1.set_ylabel('Applications', fontsize=12)
ax1.grid(True, alpha=0.3)
ax1.axvline(week_1, color='red', linestyle='--', alpha=0.5, label='Period 1')
ax1.axvline(week_2, color='green', linestyle='--', alpha=0.5, label='Period 2')
ax1.legend()

# Bookings trend
ax2.plot(weekly_totals['week_begin_date'], weekly_totals['num_tot_bks'], 
         marker='o', markersize=3, linewidth=1.5, color='#A23B72')
ax2.set_title('Weekly Bookings Trend', fontsize=14, fontweight='bold')
ax2.set_ylabel('Bookings', fontsize=12)
ax2.set_xlabel('Week', fontsize=12)
ax2.grid(True, alpha=0.3)
ax2.axvline(week_1, color='red', linestyle='--', alpha=0.5, label='Period 1')
ax2.axvline(week_2, color='green', linestyle='--', alpha=0.5, label='Period 2')
ax2.legend()

plt.tight_layout()
plt.show()

## 10. Export Results (Optional)

Results can be exported using pandas if needed:

In [ ]:
# Export functionality
# results.summary.to_csv('symmetric_summary_weekly.csv', index=False)
# results.segment_detail.to_csv('symmetric_segment_detail_weekly.csv', index=False)

## Summary

This weekly analysis demonstrates:

1. **Weekly granularity**: More frequent data points for detailed trend analysis
2. **Symmetric decomposition**: Order-independent methodology works seamlessly with weekly data
3. **NEW: date_column parameter**: Clean, explicit code without column renaming
4. **Same insights**: All the same dimensional breakdowns and effect calculations
5. **Enhanced visibility**: Weekly data allows for more timely identification of changes

### Key Advantages of Weekly Data:
- Faster detection of trends and anomalies
- More granular understanding of performance changes
- Better alignment with operational decision-making cycles
- Reduced aggregation bias from monthly averaging

### Considerations:
- Higher data volume (4x more data points than monthly)
- More volatility in weekly metrics
- May require smoothing for long-term trend analysis

### Using the date_column Parameter:

```python
# For weekly data
results = calculate_decomposition(
    df=df,
    date_a='2024-01-01',
    date_b='2024-01-08',
    lender='ACA',
    date_column='week_begin_date'  # Specify your date column
)

# For monthly data (default)
results = calculate_decomposition(
    df=df,
    date_a='2023-06-01',
    date_b='2024-06-01',
    lender='ACA'  # Uses 'month_begin_date' by default
)

# For custom date columns
results = calculate_decomposition(
    df=df,
    date_a='2023-06-01',
    date_b='2024-06-01',
    lender='ACA',
    date_column='my_custom_date_col'
)
```